# Bokeh: quick (and partial) summary - part 2
Marco Chierici & Giuseppe Jurman

May, 25th 2022

(partially abridged from [Data Visualisation with Bokeh](https://github.com/ernestoarbitrio/bokeh-data-visualisation) and the [Bokeh User Guide](https://docs.bokeh.org/en/latest/docs/user_guide.html))

In [1]:
from bokeh.plotting import figure, output_notebook, show
import pandas as pd
import numpy as np
output_notebook()

Loading BokehJS ...

## Annotations and Plot Tools

Bokeh comes with a number of interactive tools. There are three categories of tool interactions:

    Gestures:
        Pan/Drag Tools
        Click/Tap Tools
        Scroll/Pinch Tools
    Actions
    Inspectors

For each type of gesture, one tool can be active at any given time, and the active tool is indicated on the toolbar by a highlight next to to the tool icon. Actions are immediate or modal operations that are only activated when their button in the toolbar is pressed. Inspectors are passive tools that report information or annotate the plot in some way.

**Positioning the Toolbar**

By default, Bokeh plots come with a toolbar above the plot. In this section you will learn how to specify a different location for the toolbar, or to remove it entirely.

The toolbar location can be specified by passing the toolbar_location parameter to the figure() function. Valid values are:

    "above"
    "below"
    "left"
    "right"

If you would like to hide the toolbar entirely, pass None.

Below is some code that positions the toolbar below the plot. Try running the code and changing the toolbar_location value.

In [2]:
# create a new plot with the toolbar below
p = figure(width=400, height=400,
           title=None, toolbar_location="below")
p.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=10)
show(p)

## Spans

`Spans` are "infinite" vertical or horizonal lines. When creating them, you specify the `dimension` that should be spanned (i.e., `width` or `height`), any visual line properties for the appearance, and the location along the dimension where the line should be drawn. Let's look at an example that adds two horizontal spans to a simple plot:

In [3]:
from bokeh.models.annotations import Span

x = np.linspace(0, 20, 200)
y = np.sin(x)

p = figure(y_range=(-2, 2))
p.line(x, y)

upper = Span(location=1, dimension='width', line_color='olive', 
             line_width=4)
p.add_layout(upper)

lower = Span(location=-1, dimension='width', line_color='firebrick', line_width=4)
p.add_layout(lower)

show(p)

Try re-running the code and changing one or both lines to vertical.

## Box Annotations

Sometimes you might want to call out some region of the plot by drawing a shaded box. This can be done with the `BoxAnnotation`, which is configured with the coordinate properties:
* `top`
* `left`
* `bottom`
* `right`

as well as any visual line or fill properties to control the appearance. 

"Infinite" boxes can be made by leaving any of the coordinates unspecified. E.g., if `top` is not given, the box will always extend to the top of the plot area, regardless of any panning or zooming that happens.

Let's take a look at an example that adds a few shaded boxes to a plot:

In [4]:
from bokeh.models.annotations import BoxAnnotation

x = np.linspace(0, 20, 200)
y = np.sin(x)

p = figure(y_range=(-2, 2))
p.line(x, y)

# "infinite" box that always fills the top of the plot
upper = BoxAnnotation(bottom=1, fill_alpha=0.1, fill_color='olive')
p.add_layout(upper)

# "infinite" box that always fills the bottom of the plot
lower = BoxAnnotation(top=-1, fill_alpha=0.1, fill_color='firebrick')
p.add_layout(lower)

# a finite box
center = BoxAnnotation(top=0.6, bottom=-0.3, left=7, right=12, fill_alpha=0.1, fill_color='navy')
p.add_layout(center)

show(p)

## Label

The `Label` annotation allows you to easily attach single text labels to plots. The position and text to display are configured as `x`, `y`, and `text`:

```python
Label(x=10, y=5, text="Some Label")
```

By default the units are in "data space" but `x_units` and `y_units` may be set to `"screen"` to position the label relative to the canvas. Labels can also accept `x_offset` and `y_offset` to offset the final position from `x` and `y` by a given screen space distance. 

`Label` objects also have standard text, line (`border_line`) and fill (`background_fill`) properties. The line and fill properties apply to a bounding box around the text:

```python
Label(x=10, y=5, text="Some Label", text_font_size="12pt", 
      border_line_color="red", background_fill_color="blue")
```

In [5]:
from bokeh.models.annotations import Label

p = figure(x_range=(0,10), y_range=(0,10))
p.circle([2, 5, 8], [4, 7, 6], color="olive", size=10)

label = Label(x=5, y=7, x_offset=12, text="Second Point", 
              text_baseline="middle")
p.add_layout(label)

show(p)

## LabelSet

The `LabelSet` annotation allows you to create many labels at once, for instance if you want to label an entire set of scatter markers. They are similar to `Label`, but they can also 
accept a `ColumnDataSource` as the `source` property, and then `x` and `y` may refer to columns in the data source, e.g. `x="col2"` (but may also still be fixed values, e.g. `x=10`).

In [6]:
from bokeh.models import ColumnDataSource, LabelSet

source = ColumnDataSource(data=dict(
    temp=[166, 171, 172, 168, 174, 162],
    pressure=[165, 189, 220, 141, 260, 174],
    names=['A', 'B', 'C', 'D', 'E', 'F']))

p = figure(x_range=(160, 175))
p.scatter(x='temp', y='pressure', size=8, source=source)
p.xaxis.axis_label = 'Temperature (C)'
p.yaxis.axis_label = 'Pressure (lbs)'

labels = LabelSet(x='temp', y='pressure', text='names', level='glyph',
                  x_offset=5, y_offset=5, 
                  source=source, render_mode='canvas')


p.add_layout(labels)

show(p)

## Arrows

The `Arrow` annotation allows you to "point" at different things on your plot, and can be especially useful in conjuction with labels. 

For example, to create an arrow that points from `(0,0)` to `(1,1)`:

```python
p.add_layout(Arrow(x_start=0, y_start=0, x_end=1, y_end=1))
```

This arrow will have the default [`OpenHead`](https://docs.bokeh.org/en/latest/docs/reference/models/arrow_heads.html#openhead) arrow head at the end of the arrow. Other kinds of arrow heads include [`NormalHead`](https://docs.bokeh.org/en/latest/docs/reference/models/arrow_heads.html#normalhead) and [`VeeHead`](https://docs.bokeh.org/en/latest/docs/reference/models/arrow_heads.html#veehead). The arrow head type can be controlled by setting the `start` and `end` properties of `Arrow` objects:

```python
p.add_layout(Arrow(start=OpenHead(), end=VeeHead(), 
             x_start=0, y_start=0, x_end=1, y_end=0))
```

This will create a double-ended arrow with an "open" head at the start, and a "vee" head at the end. Arrowheads have the standard set of line and fill properties to control their appearance. As an example

```python
OpenHead(line_color="firebrick", line_width=4)
```

The code and plot below shows several of these configurations together.

In [7]:
from bokeh.models.annotations import Arrow
from bokeh.models.arrow_heads import OpenHead, NormalHead, VeeHead

p = figure(plot_width=600, plot_height=600)

p.circle(x=[0, 1, 0.5], y=[0, 0, 0.7], radius=0.1,
         color=["navy", "yellow", "red"], fill_alpha=0.1)

p.add_layout(Arrow(end=OpenHead(line_color="firebrick", line_width=4),
                   x_start=0, y_start=0, x_end=1, y_end=0))

p.add_layout(Arrow(end=NormalHead(fill_color="orange"),
                   x_start=1, y_start=0, x_end=0.5, y_end=0.7))

p.add_layout(Arrow(end=VeeHead(size=35), line_color="red",
                   x_start=0.5, y_start=0.7, x_end=0, y_end=0))

show(p)

## Legends

When plots have multiple glyphs, it is desirable to include a legend to help users interpret what they see. Bokeh can generate legends easily based on the glyphs that are added.

The easiest way to add a legend to your plot is to include any of the [`legend_label`](https://docs.bokeh.org/en/latest/docs/user_guide/annotations.html?highlight=legends#userguide-annotations-legends-legend-label), [`legend_group`](https://docs.bokeh.org/en/latest/docs/user_guide/annotations.html?highlight=legends#userguide-annotations-legends-legend-group), or [`legend_field`](https://docs.bokeh.org/en/latest/docs/user_guide/annotations.html?highlight=legends#userguide-annotations-legends-legend-field) properties when calling glyph methods. Bokeh then creates a Legend object for you automatically.

### Simple Legends

In the simplest case, you can simply pass a string as the `legend_label` argument to a glyph function:
```python
p.circle(x, y, legend_label="sin(x)")
``` 
In this case Bokeh will automatically create a legend that shows a representation of that glyph, labeled by the text you provided. A full example is below.

In [8]:
x = np.linspace(0, 4*np.pi, 100)
y = np.sin(x)

p = figure(height=400)

p.circle(x, y, legend_label="sin(x)")
p.line(x, 2*y, legend_label="2*sin(x)", line_dash=[4, 4], line_color="orange", line_width=2)

show(p)

### Compound legends

In the example above, we provided a different legend label to each glyph method. Sometimes, two (or more) different glyphs are used with a single data source. 
In this case, if you assign the same label name to multiple glyphs, all the glyphs will be combined into a single, compound legend item with that label.

In [9]:
p = figure()

p.circle(x, y, legend_label="sin(x)")
p.line(x, y, legend_label="sin(x)")

p.line(x, 2*y, legend_label="2*sin(x)",
       line_dash=[4, 4], line_color="orange", line_width=2)

p.square(x, 3*y, legend_label="3*sin(x)", fill_color=None, line_color="green")
p.line(x, 3*y, legend_label="3*sin(x)", line_color="green")

show(p)

## Color bars

Color bars are especially useful if we vary the color of a glyph according to some color mapping. Bokeh color bars are configured with a color mapper and added to plots with the `add_layout` method:

```python
color_mapper = LinearColorMapper(palette="Viridis256", low=data_low, high=data_high)
color_bar = ColorBar(color_mapper=color_mapper, location=(0,0))
p.add_layout(color_bar, 'right')
```

Here is a complete example that also uses the color mapper to transform the glyph color.

In [10]:
from bokeh.sampledata.autompg import autompg
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform

source = ColumnDataSource(autompg)
color_mapper = LinearColorMapper(palette="Viridis256", low=autompg.weight.min(), 
                  high=autompg.weight.max())

p = figure(x_axis_label='Horsepower', y_axis_label='MPG', tools='', 
           toolbar_location=None)
p.circle(x='hp', y='mpg', color=transform('weight', color_mapper), 
         size=20, alpha=0.6, source=autompg)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, 
                     location=(0,0), title='Weight')
p.add_layout(color_bar, 'right')

show(p)

## Configuring tools

In [11]:
# create a new plot with the toolbar below
p = figure(plot_width=400, plot_height=400,
           title=None, toolbar_location="below")

p.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=10)

show(p)

### Specifying tools
In `bokeh` you can specify wich tools include within the toolbar and which one must be active at the plot render

The Tools can be supplied conveniently with a comma-separate string containing tool shortcut names:

```python
tools = "pan,wheel_zoom,box_zoom,reset"
```
However, this method does not allow setting properties of the tools.

Finally, it is also always possible to add new tools to a plot by passing a tool object to the add_tools method of a plot. This can also be done in conjunction with the tools keyword described above:
```python
from bokeh.models import BoxSelectTool

plot = figure(tools="pan,wheel_zoom,box_zoom,reset")
plot.add_tools(BoxSelectTool(dimensions=["width"]))
```

In [12]:
from bokeh.models import BoxSelectTool

tools = "pan,wheel_zoom,box_zoom,reset" 
p = figure(plot_width=400, plot_height=400, 
           tools=tools, title=None, 
           toolbar_location="below")
p.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=10)
p.add_tools(BoxSelectTool(dimensions='height'))
show(p)

### Setting the active tool
Bokeh toolbars can have (at most) one active tool from each kind of gesture (drag, scroll, tap). By default, Bokeh will use a default pre-defined order of preference to choose one of each kind from the set of configured tools, to be active.

However it is possible to exert control over which tool is active. At the lowest bokeh.models level, this is accomplished by using the active_drag, active_inspect, active_scroll, and active_tap properties of Toolbar. These properties can take the following values:

    None — there is no active tool of this kind
    "auto" — Bokeh chooses a tool of this kind to be active (possibly none)
    a Tool instance — Bokeh sets the given tool to be the active tool

Additionally, the active_inspect tool may accept a sequence of Tool instances to be set as the active tools

In [13]:
plot = figure(tools="pan,lasso_select,box_select", 
              plot_width=400, plot_height=400, 
              active_drag="lasso_select")
plot.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=10)
show(plot)

## Tooltips

### Basic tooltips

By default, the hover tool will generate a “tabular” tooltip where each row contains a label, and its associated value. The labels and values are supplied as a list of (label, value) tuples. For instance, the tooltip below was created with the accompanying tooltips definition below.

<img src="http://bokeh.pydata.org/en/latest/_images/hover_basic.png">

```python
	
hover.tooltips = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("radius", "@radius"),
    ("fill color", "$color[hex, swatch]:fill_color"),
    ("fill color", "$color[hex]:fill_color"),
    ("fill color", "$color:fill_color"),
    ("fill color", "$swatch:fill_color"),
    ("foo", "@foo"),
    ("bar", "@bar"),
]
```

Field names that begin with $ are special fields. These often correspond to values that are intrinsic to the plot, such as the coordinates of the mouse in data or screen space. These special fields are listed here:

 - `$index`: index of selected point in the data source
 - `$name`: value of the name property of the hovered glyph renderer
 - `$x`: x-coordinate under the cursor in data space
 - `$y`: y-coordinate under the cursor in data space
 - `$sx`: x-coordinate under the cursor in screen (canvas) space
 - `$sy`: y-coordinate under the cursor in screen (canvas) space
 - `$name`: The name property of the glyph that is hovered over
 - `$color`: colors from a data source, with the syntax: `$color[options]:field_name`. The available options are: hex (to display the color as a hex value), and swatch to also display a small color box
 - `$swatch`: color data from data source displayed as a small color box.
 
Field names that begin with @ are associated with columns in a ColumnDataSource. For instance the field name "@price" will display values from the "price" column whenever a hover is triggered. If the hover is for the 17th glyph, then the hover tooltip will correspondingly display the 17th price value.

Note that if a column name contains spaces, it must be surrounded by curly braces, e.g. @{adjusted close} will display values from a column named "adjusted close".

Sometimes (especially with stacked charts) it is desirable to allow the name of the column be specified indirectly. The field name `@$name` is distinguished in that it will look up the name field on the hovered glyph renderer, and use that value as the column name. For instance, if a user hovers on the name "US East", then `@$name` is equivalent to `@{US East}`.

Here is a complete example of how to configure and use the hover tool by setting the tooltips argument to figure:

In [14]:
from bokeh.models import ColumnDataSource

source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    desc=['A', 'b', 'C', 'd', 'E'],
))

TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("desc", "@desc"),
]

p = figure(plot_width=400, plot_height=400, tooltips=TOOLTIPS,
           title="Mouse over the dots")

p.circle('x', 'y', size=20, source=source)

show(p)

### Formatting Tooltip Fields
By default, values for fields (e.g. @foo) are displayed in a basic numeric format. However it is possible to control the formatting of values more precisely. Fields can be modified by appending a format specified to the end in curly braces. Some examples are below.

```python
"@foo{0,0.000}"    # formats 10000.1234 as: 10,000.123

"@foo{(.00)}"      # formats -10000.1234 as: (10000.123)

"@foo{($ 0.00 a)}" # formats 1230974 as: $ 1.23 m
```

In [15]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.sampledata.stocks import AAPL # Sample stock data [bokeh built-in]

def datetime(x):
    return np.array(x, dtype=np.datetime64)

source = ColumnDataSource(data={
    'date'      : datetime(AAPL['date'][::10]),
    'adj close' : AAPL['adj_close'][::10],
    'volume'    : AAPL['volume'][::10],
})

p = figure(plot_height=250, x_axis_type="datetime", tools="", toolbar_location=None,
           title="Hover Tooltip Formatting", sizing_mode="scale_width")
p.background_fill_color="#f5f5f5"
p.grid.grid_line_color="white"
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'
p.axis.axis_line_color = None

p.line(x='date', y='adj close', line_width=2, 
       color='#ebbd5b', source=source)

p.add_tools(HoverTool(
    tooltips=[
        ( 'date',   '@date{%F}'            ),
        ( 'close',  '$@{adj close}{%0.2f}' ), # use @{ } for field names with spaces
        ( 'volume', '@volume{0.00 a}'      ),
    ],

    formatters={
        '@date'        : 'datetime', # use 'datetime' formatter for 'date' field
        '@{adj close}' : 'printf',   # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    },

    # display a tooltip whenever the cursor is vertically in line with a glyph
    mode='vline'
))

show(p)

### Custom HTML Tooltip

It is also possible to supply a custom HTML template for a tooltip. To do this, pass an HTML string, with the Bokeh tooltip field name symbols wherever substitutions are desired. Note that you can also use the {safe} format after the column name to disable the escaping of HTML in the data source. An example is shown below:

In [16]:
source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[2, 5, 8, 2, 7],
    desc=['A', 'b', 'C', 'd', 'E'],
    imgs=[
        'https://bokeh.pydata.org/static/snake.jpg',
        'https://bokeh.pydata.org/static/snake2.png',
        'https://bokeh.pydata.org/static/snake3D.png',
        'https://bokeh.pydata.org/static/snake4_TheRevenge.png',
        'https://bokeh.pydata.org/static/snakebite.jpg'
    ],
    fonts=[
        '<i>italics</i>',
        '<pre>pre</pre>',
        '<b>bold</b>',
        '<small>small</small>',
        '<del>del</del>'
    ]
))

TOOLTIPS = """
    <div>
        <div>
            <img
                src="@imgs" height="42" alt="@imgs" width="42"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">
            @desc</span>
            <span style="font-size: 15px; color: #966;">[$index]</span>
        </div>
        <div>
            <span>@fonts{safe}</span>
        </div>
        <div>
            <span style="font-size: 15px;">Location</span>
            <span style="font-size: 10px; color: #696;">($x, $y)</span>
        </div>
    </div>
"""

p = figure(plot_width=400, plot_height=400, tooltips=TOOLTIPS,
           title="Mouse over the dots")

p.circle('x', 'y', size=20, source=source)

show(p)

## Gridplots and linked charts

The `gridplot()` function can be used to arrange Bokeh Plots in grid layout.

### Gridplots
`gridplot()` allows the creation of a N x M layout to visualise several plots in an ordered way. It also collects all tools into a single toolbar, and the currently active tool is the same for all plots in the grid. **It is possible to leave “empty” spaces in the grid by passing None instead of a plot object.**

In [17]:
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3

In [18]:
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# create three plots
p1 = figure(plot_width=250, plot_height=250, title=None)
p1.circle(x, y0, size=3, color=Viridis3[0])

p2 = figure(plot_width=250, plot_height=250, title=None)
p2.circle(x, y1, size=3, color=Viridis3[1])

p3 = figure(plot_width=250, plot_height=250, title=None)
p3.circle(x, y2, size=3, color=Viridis3[2])

# make a grid
grid = gridplot([[p1, p2], [None, p3]])

# show the results
show(grid)

For convenience you can also just pass a list of plots, and specify the number of columns you want in your grid. For example,

`gridplot([[s1, s2], [s3, None]])`

and

`gridplot([s1, s2, s3], ncols=2)`

Furthermore, you can specify the size of the plots in the gridplot istruction.

In [19]:
from bokeh.io import show
from bokeh.layouts import gridplot
from bokeh.palettes import Viridis3

N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# create three plots
s1 = figure()
s1.circle(x, y0, size=3, color=Viridis3[0])
s2 = figure()
s2.circle(x, y1, size=3, color=Viridis3[1])
s3 = figure()
s3.circle(x, y2, size=3, color=Viridis3[2])

# make a grid
grid = gridplot([s1, s2, s3], ncols=2, width=250, height=250)

# show the results
show(grid)

### Linked panning and brushing

Linking together various aspects of different plots can be a useful technique for data visualization. In Bokeh, such linkages are typically accomplished by sharing some plot component between plots. Below is an example that demonstrates **linked panning** (where changing the range of one plot causes others to update) by sharing range objects between the plots. Some other things to look out for in this example:

- calling `figure()` multiple times to create multiple plots
- using `gridplot()` to arrange several plots in an array
- showing new glyphs using new glyph methods Figure.triangle and Figure.square
- hiding the toolbar by setting toolbar_location to None
- setting convenience arguments color (sets both line_color and fill_color) and alpha (sets both line_alpha and fill_alpha)


In [20]:
# prepare some data
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# create a new plot
s1 = figure(title=None)
s1.circle(x, y0, size=3, color="navy", alpha=0.5)

# NEW: create a new plot and share both ranges
s2 = figure(x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.triangle(x, y1, size=3, color="firebrick", alpha=0.5)

# NEW: create a new plot and share only one range
s3 = figure(x_range=s1.x_range, title=None)
s3.square(x, y2, size=3, color="olive", alpha=0.5)

# NEW: put the subplots in a gridplot
p = gridplot([[s1, s2, s3]], toolbar_location=None, width=250, height=250)

# show the results
show(p)

Another linkage that is often useful is **linked brushing** (where a selection on one glyph causes a selection to update on all other glyphs that share the same source). Below is an example that demonstrates linked brushing between circle glyphs on two different figure() calls sharing a [ColumnDataSource](https://docs.bokeh.org/en/latest/docs/reference/models/sources.html#columndatasource):

In [21]:
x = list(range(-20, 21))
y0 = [abs(xx) for xx in x]
y1 = [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "box_select,lasso_select,reset,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

Linked selection with filtered data:

In [22]:
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter

x = list(range(-20, 21))
y0 = [abs(xx) for xx in x]
y1 = [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

# create a view of the source for one plot to use
view = CDSView(source=source, filters=[BooleanFilter([True if y > 250 or y < 100 else False for y in y1])])

TOOLS = "box_select,lasso_select,reset"

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
left.circle('x', 'y0', size=10, hover_color="firebrick", source=source)

# create another new plot, add a renderer that uses the view of the data source
right = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
right.circle('x', 'y1', size=10, hover_color="firebrick", source=source, view=view)

p = gridplot([[left, right]])

show(p)

# Interaction with widgets and JS CallBacks

## Adding Widgets

Widgets are interactive controls that can be added to Bokeh applications to provide a front end user interface to a visualization. They can drive new computations, update plots, and connect to other programmatic functionality.
Widgets can be added directly to the document root or nested inside a layout. There are two ways to program a widget’s functionality:

- Use the `CustomJS` callback. This will work in standalone HTML documents.
- Use `bokeh serve` to start the Bokeh server and set up event handlers. 

### Buttons

In [23]:
from bokeh.layouts import column
from bokeh.models import Button

button_warning = Button(label="Foo Warning", button_type="warning")
button_success = Button(label="Foo Success", button_type="success")

show(column(button_warning, button_success))

### Checkbox button group

In [24]:
from bokeh.models import CheckboxButtonGroup

checkbox_button_group = CheckboxButtonGroup(
        labels=["Option 1", "Option 2", "Option 3"], 
        active=[1, 2]) # active param show selected the slice you specify

show(checkbox_button_group)

### Checkbox group

In [25]:
from bokeh.models import CheckboxGroup

checkbox_group = CheckboxGroup(
        labels=["Option 1", "Option 2", "Option 3"], active=[0, 1])

show(checkbox_group)

## DataTable

Bokeh provides a sophisticated data table widget based on SlickGrid. Note that since the table is configured with a data source object, any plots that share this data source will automatically have selections linked between the plot and the table (even in static HTML documents). 

*The data table is sortable by clicking on the column name*

In [26]:
from datetime import date
from random import randint
from bokeh.models import ColumnDataSource, DataTable, DateFormatter, TableColumn

data = dict(
        dates=[date(2017, 3, i+1) for i in range(10)],
        downloads=[randint(0, 100) for i in range(10)],
    )
source = ColumnDataSource(data)

columns = [
        TableColumn(field="dates", title="Date", formatter=DateFormatter()),
        TableColumn(field="downloads", title="Downloads"),
    ]
data_table = DataTable(source=source, columns=columns, width=400, height=280)

show(data_table)

## Sliders

The Bokeh slider can be configured with `start` and `end` values, a `step` size, an initial `value`, and a `title`:

In [27]:
from bokeh.models import Slider, RangeSlider

slider = Slider(start=0, end=10, value=3, step=.1, title="Slider")
range_slider = RangeSlider(start=0, end=10, value=(5,9), step=.1, title="Range Slider")

show(column(slider, range_slider))

## Tab panes

In [28]:
from bokeh.models import Panel, Tabs

p1 = figure(plot_width=300, plot_height=300)
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="red", alpha=0.5)
tab1 = Panel(child=p1, title="circle")

p2 = figure(plot_width=300, plot_height=300)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="blue", alpha=0.5)
tab2 = Panel(child=p2, title="line")

tabs = Tabs(tabs=[tab1, tab2])

show(tabs)

## JavaScript callbacks

While the main goal of Bokeh is to provide a path to create rich interactive visualizations in the browser, purely from Python, there will always be specialized use-cases that are outside the capabilities of the core library. 

For this reason, Bokeh provides different ways for users to supply custom JavaScript when necessary, so that users may add custom or specialized behaviours in response to property changes and other events.
To supply a snippet of JavaScript code that should be executed (in the browser) when some event occurs, use the `CustomJS model`:

```python
from bokeh.models.callbacks import CustomJS

callback = CustomJS(args=dict(xr=plot.x_range), code="""

// JavaScript code goes here

var a = 10;

// the model that triggered the callback is cb_obj:
var b = cb_obj.value;

// models passed as args are automagically available
xr.start = a;
xr.end = b;

""")
```

### Simple CustomJS callback

Yuu can attach a custom callback to a click event (e.g., a button click) using the `js_on_click` method of Bokeh models.

To see what happens in the next example, you need to open your browser's console (usually, right-click on anywhere on the page, select "Inspect" and then choose the Console from the dashboard that opens up)

In [29]:
from bokeh.models import Button, CustomJS

button = Button(label="Foo", button_type="success")
callback = CustomJS(code="console.log('button: click!', this.toString())")
button.js_on_click(callback)

show(button)

### CustomJS for Model Property Events

These CustomJS callbacks can also be attached to property change events on any Bokeh model, using the `js_on_change` method of Bokeh models:

The below example shows how to attach a CustomJS callback to a Slider widget, so that whenever the slider value updates, the callback is executed to update the data.

In [30]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import figure

x = [x*0.005 for x in range(0, 200)]
y = x

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    const data = source.data;
    const f = cb_obj.value
    const x = data['x']
    const y = data['y']
    for (let i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    source.change.emit();
""")

slider = Slider(start=0.1, end=4, value=1, step=.1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)

show(layout)

### CustomJS for Selections
Bokeh also provides the means to specify the same kind of callback to be executed whenever a selection changes. 

As a simple demonstration, the example below simply copies selected points on the first plot to the second. However, more sophisticated actions and computations are easily constructed in a similar way.

In [31]:
from random import random
from bokeh.layouts import row
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.plotting import figure

x = [random() for x in range(500)]
y = [random() for y in range(500)]

s1 = ColumnDataSource(data=dict(x=x, y=y))
p1 = figure(plot_width=400, plot_height=400, tools="lasso_select,reset", title="Select Here")
p1.circle('x', 'y', source=s1, alpha=0.6)

s2 = ColumnDataSource(data=dict(x=[], y=[]))
p2 = figure(plot_width=400, plot_height=400, x_range=(0, 1), y_range=(0, 1),
            tools="", title="Watch Here")
p2.circle('x', 'y', source=s2, alpha=0.6)

s1.selected.js_on_change('indices', CustomJS(args=dict(s1=s1, s2=s2), code="""
        const inds = cb_obj.indices;
        const d1 = s1.data;
        const d2 = s2.data;
        d2['x'] = []
        d2['y'] = []
        for (let i = 0; i < inds.length; i++) {
            d2['x'].push(d1['x'][inds[i]])
            d2['y'].push(d1['y'][inds[i]])
        }
        s2.change.emit();
    """)
)

layout = row(p1, p2)

show(layout)

# Mapping Geo Data

Bokeh supports working with geographical data and creating map-based visualizations.

## Tile Provider Maps

Bokeh plots can consume [XYZ](https://developers.planet.com/docs/basemaps/tile-services/xyz/) tile services that use the [Web Mercator](https://en.wikipedia.org/wiki/Web_Mercator_projection) projection, which is a variant of the Mercator projection used prominently in web-based applications (such as Google Maps, which adopted it in 2005). 

The module [`bokeh.tile_providers`](https://docs.bokeh.org/en/latest/docs/reference/tile_providers.html#bokeh-tile-providers) contains several pre-configured tile sources with appropriate attribution. You can add these to a plot using the add_tile() method.

In [32]:
# reimport modules, just in case you start from this cell
from bokeh.plotting import figure, show, output_notebook
# import map-related modules
from bokeh.tile_providers import get_provider, CARTODBPOSITRON, CARTODBPOSITRON_RETINA, OSM, ESRI_IMAGERY

output_notebook()

Loading BokehJS ...

In [33]:
# range bounds supplied in web mercator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(get_provider(CARTODBPOSITRON))

show(p)

In [34]:
# range bounds supplied in web mercator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(get_provider(ESRI_IMAGERY))

show(p)

Notice also that passing `x_axis_type="mercator"` and `y_axis_type="mercator"` to `figure` generates axes with latitude and longitute labels, instead of raw Web Mercator coordinates.

### Google Maps example

To use the Google Maps API through `gmap()`, you need a valid Google API key: see https://developers.google.com/maps/documentation/javascript/get-api-key

In [35]:
import os
from bokeh.models import GMapOptions
from bokeh.plotting import gmap
from bokeh.models import ColumnDataSource

map_options = GMapOptions(lat=39.2, lng=9.11, map_type="roadmap", zoom=9)

# replace the value below with your personal API key:
api_key = os.environ["GOOGLE_API_KEY"]
# !!!Enabling Google Maps requires APIs activation on you Google Account!!!
p = gmap(api_key, map_options, title="Google Maps", width=800)

data = dict(lat=[ 39.29,  39.4,  39.1],
            lon=[  9.10,   9.3,   8.9])

p.circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, source=data)
show(p)

Note that any use of Bokeh with Google Maps must be within Google's Terms of Service.

### GeoJSON data

[GeoJSON](http://geojson.org) is a popular open standard for representing geographical features with JSON. It describes points, lines, and polygons (called Patches in Bokeh) as a collection of features. Each feature can also have a set of properties.

You can use `GeoJSONDataSource` almost seamlessly in place of Bokeh's `ColumnDataSource`.

In [36]:
from bokeh.sampledata.sample_geojson import geojson
geojson

'{"type":"FeatureCollection","features":[{"type":"Feature","id":"463098","geometry":{"type":"Point","coordinates":[-2.1208465099334717,51.4613151550293]},"properties":{"OrganisationCode":"Q64","OrganisationType":"Area Team","SubType":"UNKNOWN","OrganisationStatus":"Visible","IsPimsManaged":"True","OrganisationName":"Bath, Gloucestershire, Swindon And Wiltshire Area Team","Address1":"1st Floor","Address2":"Bewley House","Address3":"Marshfield Road","City":"Chippenham","County":"Wiltshire","Postcode":"SN15 1JW","Phone":"0113 8251 500","Email":"england.contactus@nhs.net","Website":"http://www.england.nhs.uk/south/south/bgsw-at/"}},{"type":"Feature","id":"463099","geometry":{"type":"Point","coordinates":[-2.5929524898529053,51.459877014160156]},"properties":{"OrganisationCode":"Q65","OrganisationType":"Area Team","SubType":"UNKNOWN","OrganisationStatus":"Visible","IsPimsManaged":"True","OrganisationName":"Bristol, North Somerset, Somerset And South Gloucestershire Area Team","Address1":"So

In [37]:
import json

data = json.loads(geojson)
data['features'][0]

{'type': 'Feature',
 'id': '463098',
 'geometry': {'type': 'Point',
  'coordinates': [-2.1208465099334717, 51.4613151550293]},
 'properties': {'OrganisationCode': 'Q64',
  'OrganisationType': 'Area Team',
  'SubType': 'UNKNOWN',
  'OrganisationStatus': 'Visible',
  'IsPimsManaged': 'True',
  'OrganisationName': 'Bath, Gloucestershire, Swindon And Wiltshire Area Team',
  'Address1': '1st Floor',
  'Address2': 'Bewley House',
  'Address3': 'Marshfield Road',
  'City': 'Chippenham',
  'County': 'Wiltshire',
  'Postcode': 'SN15 1JW',
  'Phone': '0113 8251 500',
  'Email': 'england.contactus@nhs.net',
  'Website': 'http://www.england.nhs.uk/south/south/bgsw-at/'}}

In [38]:
data['features'][0]['properties']

{'OrganisationCode': 'Q64',
 'OrganisationType': 'Area Team',
 'SubType': 'UNKNOWN',
 'OrganisationStatus': 'Visible',
 'IsPimsManaged': 'True',
 'OrganisationName': 'Bath, Gloucestershire, Swindon And Wiltshire Area Team',
 'Address1': '1st Floor',
 'Address2': 'Bewley House',
 'Address3': 'Marshfield Road',
 'City': 'Chippenham',
 'County': 'Wiltshire',
 'Postcode': 'SN15 1JW',
 'Phone': '0113 8251 500',
 'Email': 'england.contactus@nhs.net',
 'Website': 'http://www.england.nhs.uk/south/south/bgsw-at/'}

In [39]:
from bokeh.models import GMapOptions
from bokeh.plotting import gmap, figure
from bokeh.models import ColumnDataSource, GeoJSONDataSource

# Add custom colors to the data
for i in range(len(data['features'])):
    data['features'][i]['properties']['Color'] = ['blue', 'red'][i % 2]

# create tooltips
TOOLTIPS = [
    ('Organisation', '@OrganisationName'),
    ('City', '@City')
]

map_options = GMapOptions(lat=52, lng=-1.5, map_type="roadmap", zoom=7)
p = gmap(api_key, map_options, title="Google Maps", width=800, height=800, tooltips=TOOLTIPS)

geo_source = GeoJSONDataSource(geojson=json.dumps(data))

p.circle(x="x", y="y", size=15, color='Color', fill_alpha=0.8, source=geo_source)
show(p)

## WMTS Tile Source

WTMS is the most common web standard for tiled map data, i.e. maps supplied as standard-sized image patches from which the overall map can be constructed at a given zoom level. 

WTMS uses Web Mercator format, measuring distances from Greenwich, England as meters North and meters West, which is easy to compute but does distort the global shape. 

First let's create an empty Bokeh plot covering the USA, with bounds specified in meters:

In [40]:
from bokeh.plotting import figure
from bokeh.models import WMTSTileSource

# web mercator coordinates
USA = x_range,y_range = ((-13884029, -7453304), (2698291, 6455972))

p = figure(tools='pan, wheel_zoom, reset', x_range=x_range, y_range=y_range, 
           x_axis_type="mercator", y_axis_type="mercator")

A few WTMS tile sources are already defined in `bokeh.tile_providers`, but here we'll show how to specify the interface using a format string showing Bokeh how to request a tile with the required zoom, x, and y values from a given tile provider:

In [41]:
url = 'http://a.basemaps.cartocdn.com/rastertiles/voyager/{Z}/{X}/{Y}.png'
attribution = "Tiles by Carto, under CC BY 3.0. Data by OSM, under ODbL"

p.add_tile(WMTSTileSource(url=url, attribution=attribution))

TileRenderer(id='12396', ...)

If you show the figure, you can then use the wheel zoom and pan tools to navigate over any zoom level, and Bokeh will request the appropriate tiles from the server and insert them at the correct locations in the plot:

In [42]:
show(p)

That's all it takes to put map data into your plot!  Of course, you'll usually want to show other data as well, or you could just use the tile server's own web address. You can now add anything you would normally use in a Bokeh plot, as long as you can obtain coordinates for it in Web Mercator format.  For example:

In [43]:
def wgs84_to_web_mercator(df, lon="lon", lat="lat"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["xm"] = df[lon] * (k * np.pi/180.0)
    df["ym"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

df = pd.DataFrame(dict(name=["Austin", "NYC"], lon=[-97.7431, -74.0059], lat=[30.2672, 40.7128]))
wgs84_to_web_mercator(df)

,name,lon,lat,xm,ym
0,Austin,-97.7431,30.2672,-1.088071e+07,3.537942e+06
1,NYC,-74.0059,40.7128,-8.238299e+06,4.970072e+06


In [44]:
TOOLTIPS = [
    ('City', '@name')
]

src = ColumnDataSource(df)

p = figure(tools='pan, wheel_zoom, reset', x_range=x_range, y_range=y_range, 
           x_axis_type="mercator", y_axis_type="mercator",
           tooltips=TOOLTIPS)

p.add_tile(WMTSTileSource(url=url, attribution=attribution))

p.circle(x='xm', y='ym', fill_color='orange', size=10, source=src)
show(p)

# Additional resources

There are many things we haven't had time to tell you about. In general, to learn more about bokeh, the following resources will hopefully be helpful:

## Documentation


##### Main Page - https://bokeh.pydata.org/en/latest 

The main documentation front page, with links to many other resources

---

##### User's Guide - https://bokeh.pydata.org/en/latest/docs/user_guide.html

The user's guide has many top-oriented subsections, for example "Plotting with Basic Glyphs", "Configuring Plot Tools", or "Adding Interactions". Each user's guide section typically example code and corresponding live plots that demonstrate how to accomplish various tasks. 

---

##### Gallery - https://bokeh.pydata.org/en/latest/docs/gallery.html

One of the best ways to learn is to find an existing example similar to what you want, and to study it and then use it as a starting place. Starting from a known working example can often save time and effort when getting started by allowing you to make small, incremental changes and observing the outcome. The Bokeh docs have a large thumbnail gallery that links to live plots and apps with corresponding code.  


---

##### Reference Guide - https://bokeh.pydata.org/en/latest/docs/reference.html

If you are already familiar with Bokeh and have questions about specific details of the obejcts you are already using, the reference guide is a good resource for finding information. The reference guide is automatically generated from the project source code and is a complete resources for all bokeh models and their properties. 

---



##### Issue tracker - https://github.com/bokeh/bokeh/issues

The GitHub issue tracker is the place to go to submit ***bug reports*** and ***feature requests***. It it NOT the right place for general support questions (see the *General Community Support* links below).

